In [2]:
import os
import yaml
import joblib
#
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
#
import torchaudio
import torchaudio.transforms as T
import torchaudio.functional as F
#
import torchvision
from torchvision.io import read_image
from torchvision.transforms import ConvertImageDtype
#
import matplotlib.pyplot as plt
#
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [3]:
print(torch.__version__)
print(torchaudio.__version__)
print(torchvision.__version__)

2.3.0+cu121
2.3.0+cu121
0.18.0+cu121


In [4]:
raw_path = 'data/raw/'
data_path = 'data/spectrogram/'
LEARNING_RATE = 0.001

In [5]:
labels = yaml.safe_load(open('labels.yaml'))

In [6]:
X, y = zip(*labels.items())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0, shuffle=False)

In [7]:
def training(model, optimizer, criterion, dataloader):
    losses = 0
    y_true, y_score = [], []
    #
    model.train()
    for X, Y in dataloader:
        X, Y = X.to(device), Y.to(device, dtype=torch.float32)
        #
        output = model(X)
        loss = criterion(output.view(1), Y)
        #
        losses += loss.item()
        y_true.append(Y.item())
        y_score.append(torch.sigmoid(output).item())
        #
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return losses, roc_auc_score(y_true, y_score)


def testing(model, optimizer, criterion, dataloader):
    losses = 0
    y_true, y_score = [], []
    #
    model.eval()
    with torch.inference_mode():
        for X, Y in dataloader:
            X, Y = X.to(device), Y.to(device, dtype=torch.float32)
            #
            output = model(X)
            loss = criterion(output.view(1), Y)
            #
            losses += loss.item()
            y_true.append(Y.item())
            y_score.append(torch.sigmoid(output).item())
    #
    return losses, roc_auc_score(y_true, y_score)

# Modelling using images

In [8]:
device = 'cpu'

In [16]:
from utils.miscellaneous import update_stats

In [10]:
class Podcast(Dataset):

    def __init__(self, X, y):
        self.x = X
        self.y = y
        self.cid = ConvertImageDtype(torch.float32)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        label = self.y[idx]
        episode = self.x[idx]
        image = read_image(f'{data_path}/{episode}.png')
        return self.cid(image), int(label)

In [11]:
train = Podcast(X_train, y_train)
train_dataloader = DataLoader(train, shuffle=True)

In [12]:
test = Podcast(X_test, y_test)
test_dataloader = DataLoader(test)

In [13]:
class CNNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=4, out_channels=4, kernel_size=(6, 3)),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=(6, 3))
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=4, out_channels=4, kernel_size=(12, 6)),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=(12, 6))
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=4, out_channels=4, kernel_size=(18, 9)),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(output_size=(64, 8))
        )
        #
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.33)
        self.linear1 = nn.Linear(2048, 128)
        self.linear2 = nn.Linear(128, 1)

    def forward(self, x):
        shapes = [x.shape]
        # print()
        x = self.layer1(x)
        shapes.append(x.shape)
        x = self.layer2(x)
        shapes.append(x.shape)
        x = self.layer3(x)
        # shapes.append(x.shape)
        #
        x = self.flatten(x)
        # print(x.shape)
        x = self.relu(self.linear1(x))
#         x = self.dropout(x, training=self.training)
        # print(x.shape, self.training)
        #
        print('=' if self.training else '-', end='')
        # print(shapes)
        return self.linear2(x)


model = CNNet().to(device, dtype=torch.float32)

In [15]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
filename = '04_spectrogram_images'

In [ ]:
# !rm artifacts/04_spectrogram_images.joblib
# !rm artifacts/04_spectrogram_images.pt

In [15]:
if os.path.exists(f'artifacts/{filename}.pt'):
    model.load_state_dict(
        torch.load(f'artifacts/{filename}.pt')
    )
    print('!Model weights are loaded.')
    
if os.path.exists(f'artifacts/{filename}.joblib'):
    stats = joblib.load(f'artifacts/{filename}.joblib')
    print('!Training stats is loaded.')
else:
    stats = {
        'epoch': [],
        'training_loss': [],
        'testing_loss': [],
        'training_auc': [],
        'testing_auc': [],
    }
    
se = stats['epoch'][-1] if stats['epoch'] else 0

In [17]:
figure = get_figure(stats)
figure

FigureWidget({
    'data': [{'line': {'color': 'black', 'dash': 'dash', 'width': 0.5},
              'marker': {'color': 'white', 'line': {'color': 'black', 'width': 1}, 'size': 6},
              'mode': 'lines+markers',
              'name': 'Training',
              'type': 'scatter',
              'uid': 'd75c8fd7-84f7-4255-93c1-d86de94902d8',
              'x': [],
              'xaxis': 'x',
              'y': [],
              'yaxis': 'y'},
             {'line': {'color': 'red', 'dash': 'dash', 'width': 0.5},
              'marker': {'color': 'white', 'line': {'color': 'red', 'width': 1}, 'size': 6},
              'mode': 'lines+markers',
              'name': 'Testing',
              'type': 'scatter',
              'uid': 'a67c2412-66fe-4a2c-9ea1-250839a17e70',
              'x': [],
              'xaxis': 'x',
              'y': [],
              'yaxis': 'y'},
             {'line': {'color': 'black', 'dash': 'dash', 'width': 0.5},
              'marker': {'color': 'white', '

In [ ]:
for epoch in range(se, 201):
    train_loss, train_auc = training(model, optimizer, criterion, train_dataloader)
    test_loss, test_auc = testing(model, optimizer, criterion, test_dataloader)
    #
    update_stats(stats, epoch, train_loss, test_loss, train_auc, test_auc)
    update_figure(figure, stats)
    #
    if epoch % 5 == 0:
        joblib.dump(stats, f'artifacts/{filename}.joblib')
        torch.save(
            model.state_dict(),
            f'artifacts/{filename}.pt'
        )

=

/opt/conda/lib/python3.11/site-packages/torch/autograd/graph.py:744: UserWarning:

Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)



=============================================================================--------------------------
Epoch: 0 | Training loss: 68.0031 | Testing loss: 16.8024 | Training AUC: 0.4513 | Testing AUC: 0.3383
==============================================================================--------------------------
Epoch: 1 | Training loss: 49.2129 | Testing loss: 15.2376 | Training AUC: 0.4464 | Testing AUC: 0.6090
==============================================================================--------------------------
Epoch: 2 | Training loss: 47.3124 | Testing loss: 15.1532 | Training AUC: 0.3166 | Testing AUC: 0.6165

# Modelling `as is`

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [8]:
from utils.spectrograms import get_spectrogram
from utils.miscellaneous import update_stats

In [9]:
class Podcast(Dataset):

    def __init__(self, X, y):
        self.x = X
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        label = self.y[idx]
        episode = self.x[idx]
        #
        waveform, sample_rate = torchaudio.load(f'{raw_path}/{episode}.mp3')
        #
        return get_spectrogram(waveform.mean(dim=0), sample_rate), label

In [10]:
train = Podcast(X_train, y_train)
train_dataloader = DataLoader(train, shuffle=True)

In [11]:
test = Podcast(X_test, y_test)
test_dataloader = DataLoader(test)

In [12]:
class CNNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(6, 3)),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=(6, 3))
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(12, 6)),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=(12, 6))
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(18, 9)),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(output_size=(64, 8))
        )
        #
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.33)
        self.linear1 = nn.Linear(512, 64)
        self.linear2 = nn.Linear(64, 1)

    def forward(self, x):
        # print('.', end='')
        shapes = [x.shape]
        # print()
        x = self.layer1(x)
        shapes.append(x.shape)
        x = self.layer2(x)
        shapes.append(x.shape)
        x = self.layer3(x)
        #
        x = self.flatten(x)
        # print(x.shape)
        x = self.relu(self.linear1(x))
        # x = self.dropout(x, training=self.training)
        # print(shapes, self.training)
        print('=' if self.training else '-', end='')
        #
        return self.linear2(x)


model = CNNet().to(device, dtype=torch.float32)

In [15]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [13]:
filename = '04_spectrogram'

In [14]:
# !rm artifacts/04_spectrogram.joblib
# !rm artifacts/04_spectrogram.pt

In [15]:
if os.path.exists(f'artifacts/{filename}.pt'):
    model.load_state_dict(
        torch.load(f'artifacts/{filename}.pt')
    )
    print('!Model weights are loaded.')
    
if os.path.exists(f'artifacts/{filename}.joblib'):
    stats = joblib.load(f'artifacts/{filename}.joblib')
    print('!Training stats is loaded.')
else:
    stats = {
        'epoch': [],
        'training_loss': [],
        'testing_loss': [],
        'training_auc': [],
        'testing_auc': [],
    }
    
se = stats['epoch'][-1] if stats['epoch'] else 0

In [17]:
figure = get_figure(stats)
figure

FigureWidget({
    'data': [{'line': {'color': 'black', 'dash': 'dash', 'width': 0.5},
              'marker': {'color': 'white', 'line': {'color': 'black', 'width': 1}, 'size': 6},
              'mode': 'lines+markers',
              'name': 'Training',
              'type': 'scatter',
              'uid': 'd75c8fd7-84f7-4255-93c1-d86de94902d8',
              'x': [],
              'xaxis': 'x',
              'y': [],
              'yaxis': 'y'},
             {'line': {'color': 'red', 'dash': 'dash', 'width': 0.5},
              'marker': {'color': 'white', 'line': {'color': 'red', 'width': 1}, 'size': 6},
              'mode': 'lines+markers',
              'name': 'Testing',
              'type': 'scatter',
              'uid': 'a67c2412-66fe-4a2c-9ea1-250839a17e70',
              'x': [],
              'xaxis': 'x',
              'y': [],
              'yaxis': 'y'},
             {'line': {'color': 'black', 'dash': 'dash', 'width': 0.5},
              'marker': {'color': 'white', '

In [ ]:
for epoch in range(se, 201):
    train_loss, train_auc = training(model, optimizer, criterion, train_dataloader)
    test_loss, test_auc = testing(model, optimizer, criterion, test_dataloader)
    #
    update_stats(stats, epoch, train_loss, test_loss, train_auc, test_auc)
    update_figure(figure, stats)
    #
    if epoch % 5 == 0:
        joblib.dump(stats, f'artifacts/{filename}.joblib')
        torch.save(
            model.state_dict(),
            f'artifacts/{filename}.pt'
        )

=

/opt/conda/lib/python3.11/site-packages/torch/autograd/graph.py:744: UserWarning:

Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)



=============================================================================--------------------------
Epoch: 0 | Training loss: 68.0031 | Testing loss: 16.8024 | Training AUC: 0.4513 | Testing AUC: 0.3383
==============================================================================--------------------------
Epoch: 1 | Training loss: 49.2129 | Testing loss: 15.2376 | Training AUC: 0.4464 | Testing AUC: 0.6090
==============================================================================--------------------------
Epoch: 2 | Training loss: 47.3124 | Testing loss: 15.1532 | Training AUC: 0.3166 | Testing AUC: 0.6165

## + augmentations

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [9]:
from utils.augmentations import augment, masking
from utils.spectrograms import get_spectrogram
from utils.miscellaneous import update_stats
from utils.plot import get_figure, update_figure

In [10]:
class Podcast(Dataset):

    def __init__(self, X, y, clipping=True, training=True):
        self.x = X
        self.y = y
        self.clipping = clipping
        self.training = training

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        label = self.y[idx]
        episode = self.x[idx]
        #
        waveform, sample_rate = torchaudio.load(f'{raw_path}/{episode}.mp3')
        waveform = waveform.mean(dim=0)
        #
        if self.training:
            waveform = augment(waveform, sample_rate, self.clipping)
        spectrogram = get_spectrogram(waveform, sample_rate)
        if self.training:
            spectrogram = masking(spectrogram)
        #
        return spectrogram, label

In [11]:
train = Podcast(X_train, y_train)
train_dataloader = DataLoader(train, shuffle=True)

In [12]:
test = Podcast(X_test, y_test, training=False)
test_dataloader = DataLoader(test)

In [13]:
class CNNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(6, 3)),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=(6, 3))
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(12, 6)),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=(12, 6))
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(18, 9)),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(output_size=(64, 4))
        )
        #
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.33)
        self.linear1 = nn.Linear(256, 64)
        self.linear2 = nn.Linear(64, 1)

    def forward(self, x):
        # 
        shapes = [x.shape]
        # print()
        x = self.layer1(x)
        shapes.append(x.shape)
        x = self.layer2(x)
        shapes.append(x.shape)
        x = self.layer3(x)
        #
        x = self.flatten(x)
        # print(x.shape)
        x = self.relu(self.linear1(x))
        # x = self.dropout(x, training=self.training)
        # print(shapes, self.training)
        print('=' if self.training else '-', end='')
        #
        return self.linear2(x)


model = CNNet().to(device, dtype=torch.float32)

In [15]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [14]:
filename = '04_spectrogram_augmented'

In [ ]:
# !rm artifacts/04_spectrogram_augmented.joblib
# !rm artifacts/04_spectrogram_augmented.pt

In [15]:
if os.path.exists(f'artifacts/{filename}.pt'):
    model.load_state_dict(
        torch.load(f'artifacts/{filename}.pt')
    )
    print('!Model weights are loaded.')
    
if os.path.exists(f'artifacts/{filename}.joblib'):
    stats = joblib.load(f'artifacts/{filename}.joblib')
    print('!Training stats is loaded.')
else:
    stats = {
        'epoch': [],
        'training_loss': [],
        'testing_loss': [],
        'training_auc': [],
        'testing_auc': [],
    }
    
se = stats['epoch'][-1] if stats['epoch'] else 0

In [17]:
figure = get_figure(stats)
figure

FigureWidget({
    'data': [{'line': {'color': 'black', 'dash': 'dash', 'width': 0.5},
              'marker': {'color': 'white', 'line': {'color': 'black', 'width': 1}, 'size': 6},
              'mode': 'lines+markers',
              'name': 'Training',
              'type': 'scatter',
              'uid': 'd75c8fd7-84f7-4255-93c1-d86de94902d8',
              'x': [],
              'xaxis': 'x',
              'y': [],
              'yaxis': 'y'},
             {'line': {'color': 'red', 'dash': 'dash', 'width': 0.5},
              'marker': {'color': 'white', 'line': {'color': 'red', 'width': 1}, 'size': 6},
              'mode': 'lines+markers',
              'name': 'Testing',
              'type': 'scatter',
              'uid': 'a67c2412-66fe-4a2c-9ea1-250839a17e70',
              'x': [],
              'xaxis': 'x',
              'y': [],
              'yaxis': 'y'},
             {'line': {'color': 'black', 'dash': 'dash', 'width': 0.5},
              'marker': {'color': 'white', '

In [ ]:
for epoch in range(se, 201):
    train_loss, train_auc = training(model, optimizer, criterion, train_dataloader)
    test_loss, test_auc = testing(model, optimizer, criterion, test_dataloader)
    #
    update_stats(stats, epoch, train_loss, test_loss, train_auc, test_auc)
    update_figure(figure, stats)
    #
    if epoch % 5 == 0:
        joblib.dump(stats, f'artifacts/{filename}.joblib')
        torch.save(
            model.state_dict(),
            f'artifacts/{filename}.pt'
        )

=

/opt/conda/lib/python3.11/site-packages/torch/autograd/graph.py:744: UserWarning:

Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)



=============================================================================--------------------------
Epoch: 0 | Training loss: 68.0031 | Testing loss: 16.8024 | Training AUC: 0.4513 | Testing AUC: 0.3383
==============================================================================--------------------------
Epoch: 1 | Training loss: 49.2129 | Testing loss: 15.2376 | Training AUC: 0.4464 | Testing AUC: 0.6090
==============================================================================--------------------------
Epoch: 2 | Training loss: 47.3124 | Testing loss: 15.1532 | Training AUC: 0.3166 | Testing AUC: 0.6165
==============================================================================--------------------------
Epoch: 3 | Training loss: 46.5442 | Testing loss: 15.0735 | Training AUC: 0.5146 | Testing AUC: 0.6466